# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [59]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [60]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print("number of input files: ", len(file_path_list))

/home/claudiordgz/udacity/dend/project_2
number of input files:  30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [61]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("Total number of rows: ", len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Total number of rows:  8056


In [62]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [63]:
try: 
    cluster = cassandra.cluster.Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [64]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [65]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [66]:
tables = [
    """CREATE TABLE IF NOT EXISTS music_app_history_q1 
    (artist text,
     song text,
     length float,
     session_id int,
     item_in_session int,
     PRIMARY KEY (
         session_id,
         item_in_session
    ))""",
    """CREATE TABLE IF NOT EXISTS music_app_history_q2 
    (artist text,
     song text,
     user_name text,
     session_id int,
     item_in_session int,
     user_id int,
     PRIMARY KEY (
         session_id,
         user_id,
         item_in_session
    ))""",
    """CREATE TABLE IF NOT EXISTS music_app_history_q3 
    (user_name text,
     song text,
     PRIMARY KEY (
         song,
         user_name
    ))""",
]

for table in tables:
    try:
        session.execute(table)
    except Exception as e:
        print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

`select artist, song, length in music_app_history where sessionId=338 and itemInSession=4`

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

`select artist, song, user_name in music_app_history where sessionId=182 and userid=10`

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

`select user_name in music_app_history where song='All Hands Against His Own'`


In [67]:
def to_val(key, line):
    """Will fetch the index in the row values and cast it if necessary.
    
    Keyword arguments:
    key -- the desired key based on the DB Schema
    line -- the line of values received from the CSV
    """
    indices = {
        "artist": 0,
        "first_name": 1,
        "gender": 2,
        "item_in_session": 3,
        "last_name": 4,
        "length": 5,
        "level": 6,
        "location": 7,
        "session_id": 8,
        "song": 9,
        "user_id": 10
    }
    value = line[indices[key]]
    casts = {
        "length": float,
        "session_id": int,
        "item_in_session": int,
        "user_id": int
    }
    if key in casts:
        value = casts[key](value)
    return value

def insert_to_cassandra(query, values):
    """Executes query and raises error if it was not possible
    
    Keyword arguments:
    query -- INSERT statement
    values -- tuple of values (1,2,3,4,...,n) expected by query
    """
    try:
        session.execute(query, values)
    except Exception as e:
        print(e)
        raise e


def insert_data_for_q1(row):
    """From an Array of values of event_datafile_new.csv
       will transform into proper input and insert to cassandra
       the following.
       (artist string, song string, length float, session_id int, item_in_session int)

    Keyword arguments:
    row -- row array of values received from CSV
    """
    values = (to_val("artist", row), to_val("song", row), to_val("length", row), to_val("session_id", row), to_val("item_in_session", row))
    query = "INSERT INTO music_app_history_q1 (artist, song, length, session_id, item_in_session)"
    query = query + " VALUES (%s, %s, %s, %s, %s)"
    insert_to_cassandra(query, values)


def insert_data_for_q2(row):
    """From an Array of values of event_datafile_new.csv
       will transform into proper input and insert to cassandra
       the following.
       (artist string, song string, user_name string, session_id int, item_in_session int, user_id int)

    Keyword arguments:
    row -- row array of values received from CSV
    """
    user_name = "{} {}".format(to_val("first_name", row), to_val("last_name", row))
    values = (to_val("artist", row), to_val("song", row), user_name, to_val("session_id", row), to_val("item_in_session", row), to_val("user_id", row))
    query = "INSERT INTO music_app_history_q2 (artist, song, user_name, session_id, item_in_session, user_id)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
    insert_to_cassandra(query, values)


def insert_data_for_q3(row):
    """From an Array of values of event_datafile_new.csv
       will transform into proper input and insert to cassandra
       the following.
       (user_name string, song string)

    Keyword arguments:
    row -- row array of values received from CSV
    """
    user_name = "{} {}".format(to_val("first_name", row), to_val("last_name", row))
    values = (user_name, to_val("song", row))
    query = "INSERT INTO music_app_history_q3 (user_name, song)"
    query = query + " VALUES (%s, %s)"
    insert_to_cassandra(query, values)


In [68]:
# Load CSV data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_data_for_q1(line)
        insert_data_for_q2(line)
        insert_data_for_q3(line)

In [69]:
## Query 1

query = "select artist, song, length from music_app_history_q1 where session_id=338 and item_in_session=4"
try:
    rows = session.execute(query)
    for row in rows:
        print(row.artist, row.song, row.length)
except Exception as e:
    print(e)
    

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Do a SELECT to verify that the data have been inserted into each table

In [70]:
## Add in the SELECT statement to verify the data was entered into the table

tables = ["music_app_history_q1", "music_app_history_q2", "music_app_history_q3"]
for t in tables:
    print("GETTING RECORDS FROM TABLE: {}".format(t))
    query = "select * from {} limit 5;".format(t)
    try:
        rows = session.execute(query)
        for row in rows:
            print(row)
    except Exception as e:
        print(e)
    print()

GETTING RECORDS FROM TABLE: music_app_history_q1
Row(session_id=23, item_in_session=0, artist='Regina Spektor', length=191.08526611328125, song='The Calculation (Album Version)')
Row(session_id=23, item_in_session=1, artist='Octopus Project', length=250.95791625976562, song='All Of The Champs That Ever Lived')
Row(session_id=23, item_in_session=2, artist='Tegan And Sara', length=180.06158447265625, song='So Jealous')
Row(session_id=23, item_in_session=3, artist='Dragonette', length=153.39056396484375, song='Okay Dolores')
Row(session_id=23, item_in_session=4, artist='Lil Wayne / Eminem', length=229.58975219726562, song='Drop The World')

GETTING RECORDS FROM TABLE: music_app_history_q2
Row(session_id=23, user_id=24, item_in_session=0, artist='Regina Spektor', song='The Calculation (Album Version)', user_name='Layla Griffin')
Row(session_id=23, user_id=24, item_in_session=1, artist='Octopus Project', song='All Of The Champs That Ever Lived', user_name='Layla Griffin')
Row(session_id=23,

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [71]:
## Query 2 Ordered by itemInSession

query = "select * from music_app_history_q2 where session_id=182 and user_id=10"
try:
    rows = session.execute(query)
    for row in rows:
        print(row.artist, row.song, row.user_name, "ITEM_IN_SESSION: ", row.item_in_session)
except Exception as e:
    print(e)

                    

Down To The Bone Keep On Keepin' On Sylvie Cruz ITEM_IN_SESSION:  0
Three Drives Greece 2000 Sylvie Cruz ITEM_IN_SESSION:  1
Sebastien Tellier Kilometer Sylvie Cruz ITEM_IN_SESSION:  2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz ITEM_IN_SESSION:  3


In [72]:
## Query 3

query = "select user_name from music_app_history_q3 where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
    for row in rows:
        print(row.user_name)
except Exception as e:
    print(e)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [57]:
tables = ["music_app_history_q1", "music_app_history_q2", "music_app_history_q3"]
for t in tables:
    try:
        session.execute("drop table {}".format(t))
    except Exception as e:
        print(e)

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: ConnectionException('Pool is shutdown')})
('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: ConnectionException('Pool is shutdown')})
('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: ConnectionException('Pool is shutdown')})


### Close the session and cluster connection¶

In [58]:
session.shutdown()
cluster.shutdown()